### Metoda łokcia 
(i metryka WSS)

#### Wygenerowanie danych

In [ ]:
import pandas as pd
from sklearn.datasets import make_blobs

data = make_blobs(n_samples=1000, centers=3, cluster_std=1.0, center_box=(-4.0, 4.0), random_state=42)[0]
df = pd.DataFrame(data, columns=['x1', 'x2'])
df.head()

In [ ]:
import plotly.express as px

px.scatter(df, 'x1', 'x2', width=950, height=500, title='Algorytm K-średnich', template='plotly_dark')  # zróbmy na czarnym tle

Na początek pięć klastrów.

In [ ]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=5)
kmeans.fit(data)

In [ ]:
# wartość wss przechowywana w atrybucie inertia_

kmeans.inertia_

In [ ]:
# centroidy poszczególnych klastrów przechowywane są w atrybucie cluster_centers_

kmeans.cluster_centers_

In [ ]:
# przypisania
y_kmeans = kmeans.predict(data)
df['y_kmeans'] = y_kmeans

In [ ]:
# pięć pierwszych przypisać
df.head()

In [ ]:
# wizualizacja
px.scatter(
    df, 
    'x1', 
    'x2', 
    'y_kmeans', 
    width=950, 
    height=500, 
    title='Algorytm K-średnich - 5 klastrów', 
    template='plotly_dark'
)

Teraz pozbierajmy wartośći miary wss dla liczby klastrów od 1 do 10

In [ ]:
wss = []

for nr in range(1, 10):
    kmeans = KMeans(n_clusters=nr, n_init=10)
    kmeans.fit(data)
    wss.append(kmeans.inertia_)
    
print(wss)

In [ ]:
# Stwórzmy df na podstawie wyliczonych wartości

wss_df = pd.DataFrame(wss, columns=['wss'])
wss_df = wss_df.reset_index()  # resetuje index, tworzy nową kolumnę z indeksem do której możemy się odwoływać przez 'index'
wss_df = wss_df.rename(columns={'index': 'clusters'})
wss_df['clusters'] += 1
wss_df

I wykres na podstawie tych danych.

In [ ]:
px.line(
    wss_df, 
    x='clusters', 
    y='wss', 
    width=950, 
    height=500, 
    title='Within-Cluster-Sum of Squared Errors (WCSS)',
    template='plotly_dark'
)

W miejscu wystąpienia ostatniego wyraźnego załamania (tzw. łokcia) znajduje się liczba klastrów, która może dobrze podzielić dane. Nie jest to metoda obiektywna, trzeba dokonać wyboru, ale dobrze oddaje nam charakter zagadnienia.

W tym przypadku optymalną liczbą klastrów jest liczba 3.

Ilościowo łokieć możemy obliczyć na podstawie różnicy pomiędzy lewostronną i prawostronną pochodną. Im większa różnica tym wyraśniejszy łokieć.

#### Zwizualizujmy jeszcze rozwiązanie dla 3 klastrów

In [ ]:
# bierzemy 3 klastry, bo po 3 dalsze zwiększanie liczby klastrów nie przynosi widocznych rezultatów, dane nie skupiają się 
# wyraźnie lepiej wokół klastrów.
kmeans = KMeans(n_clusters=3, n_init=10)
kmeans.fit(data)

y_kmeans = kmeans.predict(data)
df['y_kmeans'] = y_kmeans

px.scatter(
    df, 
    'x1', 
    'x2', 
    'y_kmeans', 
    width=950, 
    height=500, 
    title='Algorytm K-średnich - 3 klastry', 
    template='plotly_dark'
)

In [ ]:
# Popatrzmy na środki klastrów
centers = pd.DataFrame(data=kmeans.cluster_centers_, columns=['c1', 'c2'])
centers

In [ ]:
# nanieśmy centroidy na wykres
import plotly.graph_objects as go

fig = px.scatter(df, 'x1', 'x2', 'y_kmeans', width=950, height=500, 
                 title='Algorytm K-średnich - 3 klastry', template='plotly_dark')
fig.add_trace(go.Scatter(x=centers['c1'], y=centers['c2'], mode='markers', 
                         marker={'size': 12, 'color': 'LightSkyBlue', 'line': {'width': 2, 'color': 'tomato'}}, 
                         showlegend=False))
fig.show()

I popatrzmy na granice decyzyjne

In [ ]:
# Do wyznaczenia granic decyzyjnych bardzo pomocna jest funkcja plot_decision_regions 
from mlxtend.plotting import plot_decision_regions
import matplotlib.pyplot as plt

plt.figure(figsize=(15, 8))
plt.title('Granice decyzyjne')
plot_decision_regions(data, y_kmeans, clf=kmeans, legend=1)  # podajemy dane, predykcja, model
# plt.scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1], s=100, c='yellow')  # centroidy

plt.show()